# Durée de retour à l'emploi : les déterminants endogènes et exogènes

## Phase exploratoire

In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.3.5'

In [29]:
import csv

1. Les données exogènes : taux de chômage et taux d'emploi

Nous avons sélectionné 2 indicateurs de l'INSEE pour "qualifier" le dynamisme d'un territoire :
- le taux de chômage trimestriel localisé par zone d'emploi : la ZE est le plus petit zonage disponible (302 zones d'emploi) et des données trimestrielles depuis 2003 (dernière série disponible au 18/05/2022 : 4ème trimestre 2021) ;
- le taux d'emploi

Il est également nécessaire d'importer une classification des zones d'emploi par commune (dont l'id est le code INSEE) afin de réaliser les jointures, et une autre classification permettant de faire la jointure entre le code INSEE et le Code Postal de la commune.

## Chargement du fichier TAUX CHOMAGE

In [3]:
!pip install xlrd

In [7]:
!pip install openpyxl

In [8]:
# d'abord il est nécessaire d'importer la librairie xlrd pour permettre à Python de lire Excel (les données INSEE ne sont pas dispobinle en csv)
import xlrd

In [9]:
import openpyxl

In [ ]:
# taux de chômage trimestriel par zone d'emploi - source INSEE
# source : https://www.insee.fr/fr/statistiques/1893230
# fichier 1 : Taux de chômage localisés par zone d'emploi, en moyenne trimestrielle : en France hors Mayotte et Guyane

chemin = ".\projet_desu_tre\data\external\chomage-zone-t1-2003-t4-2021.xlsx"
chomage_df = pd.read_excel(chemin, skiprows = 5, header = 1, sheet_name = 'txcho_ze')

# NE FONCTIONNE PAS : message d'erreur en passant par le chemin - je suis sobligée de passer par le téléchargement de la base 

In [22]:
# taux de chômage trimestriel par zone d'emploi - source INSEE
# source : https://www.insee.fr/fr/statistiques/1893230
# fichier 1 : Taux de chômage localisés par zone d'emploi, en moyenne trimestrielle : en France hors Mayotte et Guyane

chomage_df = pd.read_excel("chomage-zone-t1-2003-t4-2021.xlsx", skiprows = 4, header = 1, sheet_name = 'txcho_ze')
chomage_df.head(5)

,ZE2020,LIBZE2020,REG,LIBREG,2003-T1,2003-T2,2003-T3,2003-T4,2004-T1,2004-T2,...,2019-T3,2019-T4,2020-T1,2020-T2,2020-T3,2020-T4,2021-T1,2021-T2,2021-T3,2021-T4
0,51,Alençon,0,INTERRÉGIONAL,7.2,7.5,7.3,7.5,7.7,7.4,...,8.2,8.1,7.7,7.1,8.7,7.6,7.4,7.5,7.7,7.0
1,52,Arles,0,INTERRÉGIONAL,10.8,11.0,10.7,11.1,11.2,10.8,...,11.1,10.7,10.4,9.8,11.8,10.3,10.3,10.7,10.5,9.9
2,53,Avignon,0,INTERRÉGIONAL,9.4,9.6,9.6,9.7,9.9,9.7,...,10.6,10.3,9.8,9.0,11.3,10.1,10.1,10.1,10.3,9.6
3,54,Beauvais,0,INTERRÉGIONAL,7.1,7.3,7.3,7.8,7.9,7.8,...,8.6,8.4,8.1,7.6,9.3,8.1,8.0,8.1,8.3,7.6
4,55,Bollène-Pierrelatte,0,INTERRÉGIONAL,9.5,9.5,9.4,9.4,9.6,9.5,...,10.7,10.3,9.9,9.0,11.0,9.9,9.9,10.1,10.1,9.4


In [82]:
chomage_df.shape

(302, 80)

## Chargement du fichier CSV correspondnance Code INSEE - Code postal depuis Open Data
- Source : La Poste sur Open Data
- La base officielle des codes postaux est un jeu de données qui fournit la correspondance entre les codes postaux et les codes INSEE des communes, de France (métropole et DOM), des TOM, ainsi que de MONACO. Le fichier CSV a été mis à jour le 21 mars 2022
- URL : https://www.data.gouv.fr/fr/datasets/base-officielle-des-codes-postaux/#resources

In [55]:
codes_communes_df = pd.read_excel("laposte_hexasmal_3.xlsx")
codes_communes_df.head(5)

,code_commune_insee,nom_de_la_commune,code_postal,ligne_5,libelle_d_acheminement,coordonnees_gps
0,25620,VILLE DU PONT,25650,NaN,VILLE DU PONT,"46.999873398,6.498147193"
1,25624,VILLERS GRELOT,25640,NaN,VILLERS GRELOT,"47.361512085,6.235167025"
2,26001,SOLAURE EN DIOIS,26150,AIX EN DIOIS,SOLAURE EN DIOIS,"44.70587354,5.387770455"
3,26003,ALEYRAC,26770,NaN,ALEYRAC,"44.499150841,4.946083614"
4,26009,ANDANCETTE,26140,NaN,ANDANCETTE,"45.246625023,4.812717032"


In [83]:
codes_communes_df.shape

(39193, 3)

## Chargement du fichier base des zones d'emploi Code INSEE - ZE depuis INSEE
- Source : INSEE - composition communale des zones d'emploi (au format XLSX) - Base des zones d'emploi 2020 au 1ᵉʳ janvier 2022
- URL : https://www.insee.fr/fr/information/4652957

In [43]:
commune_ze_df = pd.read_excel("ZE2020_au_01-01-2022.xlsx", skiprows = 3, header = 1, sheet_name = 'Composition_communale')
commune_ze_df.head(5)

,Code géographique communal,Libellé géographique,Zone d'emploi 2020,Libellé zone d'emploi 2020,Partie régionale de la zone d’emploi trans-régionale,Département,Région
0,CODGEO,LIBGEO,ZE2020,LIBZE2020,ZE2020_PARTIE_REG,DEP,REG
1,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse,NaN,01,84
2,01002,L'Abergement-de-Varey,8405,Bourg en Bresse,NaN,01,84
3,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse,NaN,01,84
4,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône,NaN,01,84


In [84]:
commune_ze_df.shape

(34956, 4)

## Chargement du fichier indice de concentration de l'emploi - Observatoire des territoires
- Source : Observatoire des territoires - Nombre d'emplois pour 100 actifs occupés - division : données "communes 2021" (format xlsx)
- L'indice de concentration de l'emploi mesure le rapport entre le nombre d’emplois total proposés sur un territoire et le nombre d'actifs occupés (actifs en emploi) qui y résident. Cet indicateur permet d'apprécier la fonction de pôle d'emploi ou la fonction résidentielle d'un espace. Si cet indice est supérieur à 100 alors le nombre d'emplois proposés localement est plus important que le nombre d'actifs qui y résident et qui ont un emploi. Dans ce cas, le territoire considéré occupe une fonction de pôle d'emploi.
- URL : https://www.observatoire-des-territoires.gouv.fr/indice-de-concentration-de-lemploi

In [96]:
conc_emploi_df = pd.read_excel("insee_rp_hist_1975.xlsx", skiprows = 3, header = 1, sheet_name = 'Data')
conc_emploi_df.head(5)

,codgeo,libgeo,an,ind_concentr_emp
0,01001,L'Abergement-Clémenciat,1975,48.8
1,01001,L'Abergement-Clémenciat,1982,33.3
2,01001,L'Abergement-Clémenciat,1990,25.0
3,01001,L'Abergement-Clémenciat,1999,30.4
4,01001,L'Abergement-Clémenciat,2008,18.1


In [97]:
conc_emploi_df.shape
# là il y a un pb  244 755 lignes, c'est impossbile
#attentino, il y a plusieurs années : 1975, 1982, 1990, 1999, 008, 2013 et 2018. NE RETENIR QUE 2018

(244755, 4)

In [98]:
#attentino, il y a plusieurs années : 1975, 1982, 1990, 1999, 008, 2013 et 2018. NE RETENIR QUE 2018
#donc filtre par la colonne "an" = 2018
conc_emploi_df = conc_emploi_df.loc[conc_emploi_df['an'] == 2018].sort_values(by=['codgeo'], ascending=True)
conc_emploi_df.head(5)

,codgeo,libgeo,an,ind_concentr_emp
6,01001,L'Abergement-Clémenciat,2018,27.0
13,01002,L'Abergement-de-Varey,2018,14.6
20,01004,Ambérieu-en-Bugey,2018,132.2
27,01005,Ambérieux-en-Dombes,2018,30.6
34,01006,Ambléon,2018,12.0


In [99]:
conc_emploi_df.shape

(34965, 4)

## MERGE sur les 4 databases externes pour constituer 1 seul JDD
L'objectif est d'obtenir un data frame avec le CP + le taux de chômage + l'indice de concentration d'emploi, en concaténant les 4 JDD OPEN DATA précédents.

In [106]:
codes_communes_df.shape

(39193, 3)

In [107]:
codes_communes_df = codes_communes_df[["code_commune_insee", "nom_de_la_commune", "code_postal"]]
codes_communes.shape

(5, 3)

In [ ]:
codes_communes_df.head(5)

In [ ]:
commune_ze_df.shape

In [ ]:
pd.merge(codes_communes_df, commune_ze_df, left_on=['code_commune_insee'], right_on=['Code géographique communal']).shape

In [ ]:
# .nunique() permet de calculer le nombre de valeurs différentes (uniques)
# La ligne suivante permet donc de savoir combien il y a d'années différentes :

atmo_df["Année"].nunique()

# Ici on trouve 7 valeurs différentes.

In [ ]:
# Si l'on veut voir quelles sont ces 7 années différentes,
# On peut utiliser un .unique()
 
atmo_df["Année"].unique()